In [5]:
import litebird_sim as lbs
from astropy.time import Time
import numpy as np

start_time = 0
time_span_s = 1000.0
sampling_hz = 10.0
nside = 128

sim = lbs.Simulation(
    start_time=start_time,
    duration_s=time_span_s,
    random_seed=12345,
    imo=lbs.Imo(flatfile_location=lbs.PTEP_IMO_LOCATION) 
)


# Prendo una semplice scanning strategy dove l'asse di rotazione è allineato con l'asse Sole-Terra, e il satelitte fa un giro al minuto
sim.set_scanning_strategy(
    lbs.SpinningScanningStrategy(
        spin_sun_angle_rad=np.deg2rad(0),
        precession_rate_hz=0,
        spin_rate_hz=1 / 60,
        start_time=start_time,
    ),
    delta_time_s=5.0,
 )


# Simulo uno strumento il cui telescopio è perpendicolare all'asse di rotazione
sim.set_instrument(
    lbs.InstrumentInfo(
        boresight_rotangle_rad=0.0,
        spin_boresight_angle_rad=np.deg2rad(90),
        spin_rotangle_rad=np.deg2rad(75),
    )
)


# Un semplice detector guarda lungo la direzione del telescopio 
det = lbs.DetectorInfo(
    name="Boresight_detector",
    sampling_rate_hz=sampling_hz,
    bandcenter_ghz=100.0,
    net_ukrts=50.0,
)


sim.create_observations(detectors=det)

sim.prepare_pointings()

sky_signal = np.ones((3,12*nside*nside))*1e-4 # creazione segnale celeste

sim.fill_tods(sky_signal) # scansione mappa e riempimento TOD

sim.add_noise(noise_type='white') # aggiunta rumore strumentale bianco

# stampa primi 5 valori del TOD
for i in range(5):
    print(f"{sim.observations[0].tod[0][i]:.5e}")

6.26181e-05
3.14752e-04
2.56129e-04
2.21924e-04
1.07646e-04
